In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
import optuna
import warnings
import pandas_profiling

#importing Regressror
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Set random seed for reproducibility
np.random.seed(42)

#F Score and other metrics
from sklearn import metrics
from sklearn.metrics import  mean_absolute_error, mean_squared_error

C:\Users\youss\AppData\Local\Temp\ipykernel_14948\3402102548.py:7: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [3]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


 # Reading Train and Test datasets

In [4]:
test = pd.read_csv("C:/Users/youss/Downloads/9961_14084_bundle_archive/Test.csv")
train = pd.read_csv("C:/Users/youss/Downloads/9961_14084_bundle_archive/Train.csv")


In [5]:
train.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [6]:
test.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3
5,FDH56,9.800,Regular,0.063817,Fruits and Vegetables,117.1492,OUT046,1997,Small,Tier 1,Supermarket Type1
6,FDL48,19.350,Regular,0.082602,Baking Goods,50.1034,OUT018,2009,Medium,Tier 3,Supermarket Type2
7,FDC48,NaN,Low Fat,0.015782,Baking Goods,81.0592,OUT027,1985,Medium,Tier 3,Supermarket Type3
8,FDN33,6.305,Regular,0.123365,Snack Foods,95.7436,OUT045,2002,NaN,Tier 2,Supermarket Type1
9,FDA36,5.985,Low Fat,0.005698,Baking Goods,186.8924,OUT017,2007,NaN,Tier 2,Supermarket Type1


 # Exploratory Data Analysis

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            5681 non-null   object 
 1   Item_Weight                4705 non-null   float64
 2   Item_Fat_Content           5681 non-null   object 
 3   Item_Visibility            5681 non-null   float64
 4   Item_Type                  5681 non-null   object 
 5   Item_MRP                   5681 non-null   float64
 6   Outlet_Identifier          5681 non-null   object 
 7   Outlet_Establishment_Year  5681 non-null   int64  
 8   Outlet_Size                4075 non-null   object 
 9   Outlet_Location_Type       5681 non-null   object 
 10  Outlet_Type                5681 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 488.3+ KB


In [9]:
train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [10]:
test.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
count,4705.000000,5681.000000,5681.000000,5681.000000
mean,12.695633,0.065684,141.023273,1997.828903
std,4.664849,0.051252,61.809091,8.372256
min,4.555000,0.000000,31.990000,1985.000000
25%,8.645000,0.027047,94.412000,1987.000000
50%,12.500000,0.054154,141.415400,1999.000000
75%,16.700000,0.093463,186.026600,2004.000000
max,21.350000,0.323637,266.588400,2009.000000


### Visualising the train dataset using pandas profiling

In [11]:
# Generate the report
report = pandas_profiling.ProfileReport(train)

# Save the report to an HTML file
report.to_file('Big_Mart_Sales_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
# Display the report in the notebook
display(report)

In [13]:
#Checking for duplicate rows in the train dataset

duplicate_rows = train[train.duplicated()]
print(duplicate_rows)

Empty DataFrame
Columns: [Item_Identifier, Item_Weight, Item_Fat_Content, Item_Visibility, Item_Type, Item_MRP, Outlet_Identifier, Outlet_Establishment_Year, Outlet_Size, Outlet_Location_Type, Outlet_Type, Item_Outlet_Sales]
Index: []


In [14]:
#Checking for duplicate rows in the test dataset

duplicate_rows = test[test.duplicated()]
print(duplicate_rows)

Empty DataFrame
Columns: [Item_Identifier, Item_Weight, Item_Fat_Content, Item_Visibility, Item_Type, Item_MRP, Outlet_Identifier, Outlet_Establishment_Year, Outlet_Size, Outlet_Location_Type, Outlet_Type]
Index: []


In [15]:
# Counting the number of missing values in each column of the train dataset

missing_val = train.isnull().sum()
print(missing_val)

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64


In [16]:
# Counting the number of missing values in each column of the test dataset

missing_val = test.isnull().sum()
print(missing_val)

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64


**The 'Weight' and 'OutletSize' columns in both the train and the 'test' dataset seems to have missing values**

In [17]:
# Checking all the unique values and their counts for the train dataset   
for column in train.columns:
    print(column + ":\t" + str(train[column].nunique()) + " unique values")
    print(train[column].value_counts(), end='\n\n') 

Item_Identifier:	1559 unique values
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64

Item_Weight:	415 unique values
12.150    86
17.600    82
13.650    77
11.800    76
15.100    68
          ..
7.275      2
7.685      1
9.420      1
6.520      1
5.400      1
Name: Item_Weight, Length: 415, dtype: int64

Item_Fat_Content:	5 unique values
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

Item_Visibility:	7880 unique values
0.000000    526
0.076975      3
0.162462      2
0.076841      2
0.073562      2
           ... 
0.013957      1
0.110460      1
0.124646      1
0.054142      1
0.044878      1
Name: Item_Visibility, Length: 7880, dtype: int64

Item_Type:	16 unique values
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              85

In [18]:
# Checking all the unique values and their counts for the train dataset
for column in test.columns:
    print(column + ":\t" + str(test[column].nunique()) + " unique values")
    print(test[column].value_counts(), end='\n\n') 

Item_Identifier:	1543 unique values
DRF48    8
FDZ50    8
FDQ60    8
FDN52    8
FDW10    8
        ..
FDW49    1
FDG38    1
FDD38    1
FDA15    1
FDF04    1
Name: Item_Identifier, Length: 1543, dtype: int64

Item_Weight:	410 unique values
10.500    57
17.600    53
15.600    48
20.250    47
19.350    46
          ..
6.980      1
6.320      1
5.465      1
5.945      1
7.670      1
Name: Item_Weight, Length: 410, dtype: int64

Item_Fat_Content:	5 unique values
Low Fat    3396
Regular    1935
LF          206
reg          78
low fat      66
Name: Item_Fat_Content, dtype: int64

Item_Visibility:	5277 unique values
0.000000    353
0.060879      2
0.097213      2
0.058461      2
0.058132      2
           ... 
0.082382      1
0.026175      1
0.155144      1
0.069557      1
0.104720      1
Name: Item_Visibility, Length: 5277, dtype: int64

Item_Type:	16 unique values
Snack Foods              789
Fruits and Vegetables    781
Household                638
Frozen Foods             570
Dairy        

**The 'FatContent' column has categorical values 'LF' 'low fat' which is the same as'Low Fat' category and the value 'reg' which is same as 'Regular'. So lets convert values to either 'Low Fat' or 'Regular' based on the given data.**

In [19]:
train['Item_Fat_Content'] = train['Item_Fat_Content'].replace('LF', 'Low Fat')
train['Item_Fat_Content'] = train['Item_Fat_Content'].replace('low fat', 'Low Fat')
train['Item_Fat_Content'] = train['Item_Fat_Content'].replace('reg', 'Regular')

In [20]:
test['Item_Fat_Content'] = test['Item_Fat_Content'].replace('LF', 'Low Fat')
test['Item_Fat_Content'] = test['Item_Fat_Content'].replace('low fat', 'Low Fat')
test['Item_Fat_Content'] = test['Item_Fat_Content'].replace('reg', 'Regular')

In [21]:
train_sorted = train.sort_values(by=['Item_Identifier'])
train_sorted.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
8043,DRA12,11.6,Low Fat,0.068535,Soft Drinks,143.0154,OUT010,1998,NaN,Tier 3,Grocery Store,283.6308
118,DRA12,11.6,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,NaN,Tier 2,Supermarket Type1,2552.6772
1693,DRA12,11.6,Low Fat,0.000000,Soft Drinks,141.9154,OUT035,2004,Small,Tier 2,Supermarket Type1,992.7078
1197,DRA12,11.6,Low Fat,0.000000,Soft Drinks,141.6154,OUT045,2002,NaN,Tier 2,Supermarket Type1,3829.0158
1245,DRA12,11.6,Low Fat,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,2552.6772


In [22]:
train_sorted.tail()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
7926,NCZ54,14.65,Low Fat,0.083489,Household,161.3552,OUT049,1999,Medium,Tier 1,Supermarket Type1,7148.0288
214,NCZ54,14.65,Low Fat,0.000000,Household,161.5552,OUT010,1998,NaN,Tier 3,Grocery Store,324.9104
1896,NCZ54,NaN,Low Fat,0.082956,Household,164.0552,OUT027,1985,Medium,Tier 3,Supermarket Type3,5523.4768
7166,NCZ54,NaN,Low Fat,0.145952,Household,160.9552,OUT019,1985,Small,Tier 1,Grocery Store,162.4552
7738,NCZ54,14.65,Low Fat,0.083699,Household,163.4552,OUT018,2009,Medium,Tier 3,Supermarket Type2,2599.2832


In [23]:
test_sorted = test.sort_values(by=['Item_Identifier'])
test_sorted.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
5081,DRA12,11.60,Low Fat,0.041010,Soft Drinks,141.0154,OUT049,1999,Medium,Tier 1,Supermarket Type1
500,DRA12,11.60,Low Fat,0.040946,Soft Drinks,142.9154,OUT046,1997,Small,Tier 1,Supermarket Type1
3912,DRA12,NaN,Low Fat,0.040748,Soft Drinks,140.0154,OUT027,1985,Medium,Tier 3,Supermarket Type3
2847,DRA24,19.35,Regular,0.040091,Soft Drinks,163.6868,OUT018,2009,Medium,Tier 3,Supermarket Type2
4538,DRA24,19.35,Regular,0.039928,Soft Drinks,164.5868,OUT046,1997,Small,Tier 1,Supermarket Type1


**It can be seen that in that if we sort the dataset by 'ProductID' we can figure out the missing weight values from the given weights of the same product by looking at the weight column**

In [24]:
#Filling the missing weight values
train['Item_Weight'] = train.groupby('Item_Identifier')['Item_Weight'].apply(lambda x: x.fillna(x.mean()))

C:\Users\youss\AppData\Local\Temp\ipykernel_14948\3294396905.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  train['Item_Weight'] = train.groupby('Item_Identifier')['Item_Weight'].apply(lambda x: x.fillna(x.mean()))


In [25]:
#Filling the missing weight values
test['Item_Weight'] = test.groupby('Item_Identifier')['Item_Weight'].apply(lambda x: x.fillna(x.mean()))

C:\Users\youss\AppData\Local\Temp\ipykernel_14948\316558530.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  test['Item_Weight'] = test.groupby('Item_Identifier')['Item_Weight'].apply(lambda x: x.fillna(x.mean()))


In [26]:
train_sorted = train.sort_values(by=['Item_Identifier'])
train_sorted.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
8043,DRA12,11.6,Low Fat,0.068535,Soft Drinks,143.0154,OUT010,1998,NaN,Tier 3,Grocery Store,283.6308
118,DRA12,11.6,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,NaN,Tier 2,Supermarket Type1,2552.6772
1693,DRA12,11.6,Low Fat,0.000000,Soft Drinks,141.9154,OUT035,2004,Small,Tier 2,Supermarket Type1,992.7078
1197,DRA12,11.6,Low Fat,0.000000,Soft Drinks,141.6154,OUT045,2002,NaN,Tier 2,Supermarket Type1,3829.0158
1245,DRA12,11.6,Low Fat,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,2552.6772


In [27]:
train_sorted.tail()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
7926,NCZ54,14.65,Low Fat,0.083489,Household,161.3552,OUT049,1999,Medium,Tier 1,Supermarket Type1,7148.0288
214,NCZ54,14.65,Low Fat,0.000000,Household,161.5552,OUT010,1998,NaN,Tier 3,Grocery Store,324.9104
1896,NCZ54,14.65,Low Fat,0.082956,Household,164.0552,OUT027,1985,Medium,Tier 3,Supermarket Type3,5523.4768
7166,NCZ54,14.65,Low Fat,0.145952,Household,160.9552,OUT019,1985,Small,Tier 1,Grocery Store,162.4552
7738,NCZ54,14.65,Low Fat,0.083699,Household,163.4552,OUT018,2009,Medium,Tier 3,Supermarket Type2,2599.2832


In [28]:
test_sorted = test.sort_values(by=['Item_Identifier'])
test_sorted.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
5081,DRA12,11.60,Low Fat,0.041010,Soft Drinks,141.0154,OUT049,1999,Medium,Tier 1,Supermarket Type1
500,DRA12,11.60,Low Fat,0.040946,Soft Drinks,142.9154,OUT046,1997,Small,Tier 1,Supermarket Type1
3912,DRA12,11.60,Low Fat,0.040748,Soft Drinks,140.0154,OUT027,1985,Medium,Tier 3,Supermarket Type3
2847,DRA24,19.35,Regular,0.040091,Soft Drinks,163.6868,OUT018,2009,Medium,Tier 3,Supermarket Type2
4538,DRA24,19.35,Regular,0.039928,Soft Drinks,164.5868,OUT046,1997,Small,Tier 1,Supermarket Type1


In [29]:
test_sorted.tail()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
2318,NCZ53,9.60,Low Fat,0.024616,Health and Hygiene,190.0214,OUT017,2007,NaN,Tier 2,Supermarket Type1
1730,NCZ53,9.60,Low Fat,0.024477,Health and Hygiene,189.6214,OUT046,1997,Small,Tier 1,Supermarket Type1
2907,NCZ54,14.65,Low Fat,0.083344,Household,163.6552,OUT035,2004,Small,Tier 2,Supermarket Type1
4307,NCZ54,14.65,Low Fat,0.083290,Household,160.4552,OUT013,1987,High,Tier 3,Supermarket Type1
4700,NCZ54,14.65,Low Fat,0.083831,Household,161.4552,OUT017,2007,NaN,Tier 2,Supermarket Type1


**Analysing the columns OutletID, OutletType and OutletSize with each other so that we can figure out how to fill the missing values in the OutletSize column**

In [30]:
outlet_unique_sizes_train = train.groupby('Outlet_Identifier')['Outlet_Size'].unique()
outlet_unique_sizes_train

Outlet_Identifier
OUT010       [nan]
OUT013      [High]
OUT017       [nan]
OUT018    [Medium]
OUT019     [Small]
OUT027    [Medium]
OUT035     [Small]
OUT045       [nan]
OUT046     [Small]
OUT049    [Medium]
Name: Outlet_Size, dtype: object

In [31]:
outlet_unique_sizes_test = test.groupby('Outlet_Identifier')['Outlet_Size'].unique()
outlet_unique_sizes_test

Outlet_Identifier
OUT010       [nan]
OUT013      [High]
OUT017       [nan]
OUT018    [Medium]
OUT019     [Small]
OUT027    [Medium]
OUT035     [Small]
OUT045       [nan]
OUT046     [Small]
OUT049    [Medium]
Name: Outlet_Size, dtype: object

In [32]:
unique_sizes_train2 = train.groupby('Outlet_Identifier')['Outlet_Type'].unique()
unique_sizes_train2

Outlet_Identifier
OUT010        [Grocery Store]
OUT013    [Supermarket Type1]
OUT017    [Supermarket Type1]
OUT018    [Supermarket Type2]
OUT019        [Grocery Store]
OUT027    [Supermarket Type3]
OUT035    [Supermarket Type1]
OUT045    [Supermarket Type1]
OUT046    [Supermarket Type1]
OUT049    [Supermarket Type1]
Name: Outlet_Type, dtype: object

In [33]:
unique_sizes_test2 = test.groupby('Outlet_Identifier')['Outlet_Type'].unique()
unique_sizes_test2

Outlet_Identifier
OUT010        [Grocery Store]
OUT013    [Supermarket Type1]
OUT017    [Supermarket Type1]
OUT018    [Supermarket Type2]
OUT019        [Grocery Store]
OUT027    [Supermarket Type3]
OUT035    [Supermarket Type1]
OUT045    [Supermarket Type1]
OUT046    [Supermarket Type1]
OUT049    [Supermarket Type1]
Name: Outlet_Type, dtype: object

In [34]:
unique_sizes_train3 = train.groupby('Outlet_Type')['Outlet_Size'].unique()
unique_sizes_train3

Outlet_Type
Grocery Store                      [nan, Small]
Supermarket Type1    [Medium, High, nan, Small]
Supermarket Type2                      [Medium]
Supermarket Type3                      [Medium]
Name: Outlet_Size, dtype: object

In [35]:
unique_sizes_test3 = test.groupby('Outlet_Type')['Outlet_Size'].unique()
unique_sizes_test3

Outlet_Type
Grocery Store                      [nan, Small]
Supermarket Type1    [Medium, nan, Small, High]
Supermarket Type2                      [Medium]
Supermarket Type3                      [Medium]
Name: Outlet_Size, dtype: object

**It makes sense to fill the missing values in 'OutetSize' column whose 'OutletType' is a 'Grocery Store' with the value 'Small'.
For the missing values in 'OutletSize' column whose 'OutletType' is a 'Supermarket Type1', it is best use its mode value.**

In [36]:
# Fill missing values with 'Small' where OutletType is 'Grocery Store'
train.loc[(train['Outlet_Type'] == 'Grocery Store') & (train['Outlet_Size'].isna()), 'Outlet_Size'] = 'Small'
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [37]:
# Fill missing values with 'Small' where OutletType is 'Grocery Store'
test.loc[(test['Outlet_Type'] == 'Grocery Store') & (test['Outlet_Size'].isna()), 'Outlet_Size'] = 'Small'
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,Regular,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,Small,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,13.600,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [38]:
unique_sizes_train4 = train.groupby('Outlet_Type')['Outlet_Size'].unique()
unique_sizes_train4

Outlet_Type
Grocery Store                           [Small]
Supermarket Type1    [Medium, High, nan, Small]
Supermarket Type2                      [Medium]
Supermarket Type3                      [Medium]
Name: Outlet_Size, dtype: object

In [39]:
unique_sizes_test4 = test.groupby('Outlet_Type')['Outlet_Size'].unique()
unique_sizes_test4

Outlet_Type
Grocery Store                           [Small]
Supermarket Type1    [Medium, nan, Small, High]
Supermarket Type2                      [Medium]
Supermarket Type3                      [Medium]
Name: Outlet_Size, dtype: object

In [40]:
# Counting the number of missing values in each column of the train dataset

missing_val = train.isnull().sum()
print(missing_val)

Item_Identifier                 0
Item_Weight                     4
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1855
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64


In [41]:
# Counting the number of missing values in each column of the test dataset

missing_val = test.isnull().sum()
print(missing_val)

Item_Identifier                 0
Item_Weight                    20
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1236
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64


**Applying Label Encoding the categorical columns who have ordinal values and then apply One-Hot encoding to the categorical columns with nominal values**

In [42]:
# Initialize the Label Encoder
le = LabelEncoder()

# Fit and transform the categorical column
train['Item_Fat_Content'] = le.fit_transform(train['Item_Fat_Content'])
train['Item_Type'] = le.fit_transform(train['Item_Type'])
train['Outlet_Size'] = le.fit_transform(train['Outlet_Size'])
train['Outlet_Location_Type'] = le.fit_transform(train['Outlet_Location_Type'])
train['Outlet_Type'] = le.fit_transform(train['Outlet_Type'])
train['Item_Type'] = le.fit_transform(train['Outlet_Type'])

In [43]:
test['Item_Fat_Content'] = le.fit_transform(test['Item_Fat_Content'])
test['Item_Type'] = le.fit_transform(test['Item_Type'])
test['Outlet_Size'] = le.fit_transform(test['Outlet_Size'])
test['Outlet_Location_Type'] = le.fit_transform(test['Outlet_Location_Type'])
test['Outlet_Type'] = le.fit_transform(test['Outlet_Type'])
test['Item_Type'] = le.fit_transform(test['Outlet_Type'])

In [44]:
#product_type = train[["ProductType"]]

# Initialize the One-Hot Encoder
#ohe = OneHotEncoder()

# Fit and transform the "ProductType" column
#product_type_encoded = ohe.fit_transform(product_type).toarray()

# Create a new dataframe with the encoded values
#train_encoded = pd.DataFrame(product_type_encoded, columns=ohe.get_feature_names(['ProductType']))

# Concatenate the original dataframe with the encoded dataframe
#train = pd.concat([train, train_encoded], axis=1)

In [45]:
#product_type = test[["ProductType"]]

# Fit and transform the "ProductType" column
#product_type_encoded = ohe.fit_transform(product_type).toarray()

# Create a new dataframe with the encoded values
#test_encoded = pd.DataFrame(product_type_encoded, columns=ohe.get_feature_names(['ProductType']))

# Concatenate the original dataframe with the encoded dataframe
#test = pd.concat([test, test_encoded], axis=1)

In [46]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,0,0.016047,1,249.8092,OUT049,1999,1,0,1,3735.1380
1,DRC01,5.92,1,0.019278,2,48.2692,OUT018,2009,1,2,2,443.4228
2,FDN15,17.50,0,0.016760,1,141.6180,OUT049,1999,1,0,1,2097.2700
3,FDX07,19.20,1,0.000000,0,182.0950,OUT010,1998,2,2,0,732.3800
4,NCD19,8.93,0,0.000000,1,53.8614,OUT013,1987,0,2,1,994.7052


In [47]:
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,0,0.007565,1,107.8622,OUT049,1999,1,0,1
1,FDW14,8.300,1,0.038428,1,87.3198,OUT017,2007,3,1,1
2,NCN55,14.600,0,0.099575,0,241.7538,OUT010,1998,2,2,0
3,FDQ58,7.315,0,0.015388,1,155.0340,OUT017,2007,3,1,1
4,FDY38,13.600,1,0.118599,3,234.2300,OUT027,1985,1,2,3


**Filling the missing values in OutletSize column with its mode**

In [48]:
# Find the mode of the categorical column
mode_train = train['Outlet_Size'].mode()[0]
mode_train2 = train['Item_Weight'].mode()[0]

# Fill the missing values in the categorical column with the mode
train['Outlet_Size'].fillna(mode_train, inplace=True)
train['Item_Weight'].fillna(mode_train2, inplace=True)

In [49]:
# Find the mode of the categorical column
mode_test = test['Outlet_Size'].mode()[0]
mode_test2 = test['Item_Weight'].mode()[0]

# Fill the missing values in the categorical column with the mode
test['Outlet_Size'].fillna(mode_test, inplace=True)
test['Item_Weight'].fillna(mode_test2, inplace=True)

In [50]:
# Counting the number of missing values in each column of the train dataset

missing_val = train.isnull().sum()
print(missing_val)

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64


# Selecting the features set and target model training

In [51]:
#Selecting the desired columns for the model in 'train' dataset

X = train.drop(columns=['Item_Identifier','Item_Type','Outlet_Identifier', 'Item_Outlet_Sales'])
y = train['Item_Outlet_Sales']

In [52]:
X_test = test.drop(columns=['Item_Identifier','Item_Type','Outlet_Identifier'])

# Split the dataset into a train and validation set

In [53]:
# Split the dataset into a train and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the train and validation sets
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(6818, 8) (1705, 8) (6818,) (1705,)


# Selecting CatBoost Regressor model and training it without hyperparameter tuning

In [54]:
# Selecting CatBoostRegressor
cbr = CatBoostRegressor(verbose=False)

cbr.fit(X_train, y_train)

cbr_pred = cbr.predict(X_val)

mse = mean_squared_error(y_val, cbr_pred)
rmse = np.sqrt(mse)
print(f'Root mean squared error: {rmse:.2f}')

Root mean squared error: 1052.90


In [55]:
y_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1705 entries, 7503 to 6339
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                1705 non-null   float64
 1   Item_Fat_Content           1705 non-null   int32  
 2   Item_Visibility            1705 non-null   float64
 3   Item_MRP                   1705 non-null   float64
 4   Outlet_Establishment_Year  1705 non-null   int64  
 5   Outlet_Size                1705 non-null   int32  
 6   Outlet_Location_Type       1705 non-null   int32  
 7   Outlet_Type                1705 non-null   int32  
dtypes: float64(3), int32(4), int64(1)
memory usage: 93.2 KB


# Fine tuning the model hyperparameters using Optuna

In [56]:
# Define the objective function for Optuna
def objective(trial):
    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'depth': trial.suggest_int('depth', 1, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 10.0),
        'random_strength': trial.suggest_uniform('random_strength', 0.1, 10.0),
        'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0.0, 10.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])
    }

     # Train the model
    model = CatBoostRegressor(**params, loss_function='RMSE', verbose=False)
    model.fit(X_train , y_train)

    # Calculate the test RMSE
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    return rmse

warnings.filterwarnings("ignore")

# Set up the Optuna study and run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and RMSE
print(f"Best RMSE: {study.best_value:.4f}")
print(f"Best hyperparameters: {study.best_params}")   

[I 2023-07-06 18:14:24,630] A new study created in memory with name: no-name-fbb1343f-7dc6-437b-b686-b8ae69695e24
[I 2023-07-06 18:14:27,830] Trial 0 finished with value: 1025.1596518583706 and parameters: {'learning_rate': 0.005566011546778754, 'depth': 9, 'l2_leaf_reg': 0.02061162420933159, 'random_strength': 9.658260151808811, 'bagging_temperature': 9.285480620150214, 'grow_policy': 'Lossguide'}. Best is trial 0 with value: 1025.1596518583706.
[I 2023-07-06 18:14:46,447] Trial 1 finished with value: 1208.088972801794 and parameters: {'learning_rate': 0.23558989038937928, 'depth': 10, 'l2_leaf_reg': 0.07324101016896165, 'random_strength': 7.0142317611369664, 'bagging_temperature': 7.598440841695126, 'grow_policy': 'Depthwise'}. Best is trial 0 with value: 1025.1596518583706.
[I 2023-07-06 18:14:50,283] Trial 2 finished with value: 1215.355371187427 and parameters: {'learning_rate': 0.18430512971770352, 'depth': 7, 'l2_leaf_reg': 7.412296112727207e-05, 'random_strength': 2.34423822760

[I 2023-07-06 18:15:35,362] Trial 24 finished with value: 1019.6384344632202 and parameters: {'learning_rate': 0.010607587943151869, 'depth': 5, 'l2_leaf_reg': 0.0005074225059175248, 'random_strength': 7.659507383850127, 'bagging_temperature': 8.32418631293689, 'grow_policy': 'Lossguide'}. Best is trial 16 with value: 1018.5486556203059.
[I 2023-07-06 18:15:37,825] Trial 25 finished with value: 1019.6169968400995 and parameters: {'learning_rate': 0.011323987777673255, 'depth': 5, 'l2_leaf_reg': 0.0006631539690583163, 'random_strength': 7.517640645897279, 'bagging_temperature': 8.03054343164127, 'grow_policy': 'Lossguide'}. Best is trial 16 with value: 1018.5486556203059.
[I 2023-07-06 18:15:40,408] Trial 26 finished with value: 1038.0986402175652 and parameters: {'learning_rate': 0.02072799194949756, 'depth': 5, 'l2_leaf_reg': 0.0007273687435037472, 'random_strength': 7.469172340207782, 'bagging_temperature': 8.078044520836755, 'grow_policy': 'Lossguide'}. Best is trial 16 with value: 

[I 2023-07-06 18:16:28,380] Trial 49 finished with value: 1021.4962714958981 and parameters: {'learning_rate': 0.012521063102231172, 'depth': 6, 'l2_leaf_reg': 5.3664930610669375, 'random_strength': 6.97567555314009, 'bagging_temperature': 7.810128230076024, 'grow_policy': 'SymmetricTree'}. Best is trial 16 with value: 1018.5486556203059.
[I 2023-07-06 18:16:29,887] Trial 50 finished with value: 1020.7479803306582 and parameters: {'learning_rate': 0.008395784918870297, 'depth': 4, 'l2_leaf_reg': 7.278846811528415, 'random_strength': 8.37149578606575, 'bagging_temperature': 8.252533202030104, 'grow_policy': 'Depthwise'}. Best is trial 16 with value: 1018.5486556203059.
[I 2023-07-06 18:16:31,726] Trial 51 finished with value: 1019.9741451686168 and parameters: {'learning_rate': 0.007526830151793798, 'depth': 7, 'l2_leaf_reg': 1.0995710550508793, 'random_strength': 5.435372511775105, 'bagging_temperature': 8.49830704020595, 'grow_policy': 'SymmetricTree'}. Best is trial 16 with value: 10

[I 2023-07-06 18:17:16,356] Trial 74 finished with value: 1022.1537557975238 and parameters: {'learning_rate': 0.005761396596499072, 'depth': 5, 'l2_leaf_reg': 0.00024747560657558076, 'random_strength': 7.680154650200448, 'bagging_temperature': 8.506213230702942, 'grow_policy': 'Depthwise'}. Best is trial 16 with value: 1018.5486556203059.
[I 2023-07-06 18:17:17,822] Trial 75 finished with value: 1020.1474072155287 and parameters: {'learning_rate': 0.008998035744739038, 'depth': 4, 'l2_leaf_reg': 0.003954028283896567, 'random_strength': 8.541761338106333, 'bagging_temperature': 9.251444051598467, 'grow_policy': 'Depthwise'}. Best is trial 16 with value: 1018.5486556203059.
[I 2023-07-06 18:17:18,906] Trial 76 finished with value: 1020.8807113489501 and parameters: {'learning_rate': 0.01010423450671606, 'depth': 3, 'l2_leaf_reg': 0.0004216166955950168, 'random_strength': 6.280310725484208, 'bagging_temperature': 8.858135794685602, 'grow_policy': 'Depthwise'}. Best is trial 16 with value

[I 2023-07-06 18:18:30,947] Trial 99 finished with value: 1019.8630705499481 and parameters: {'learning_rate': 0.006132357295974064, 'depth': 7, 'l2_leaf_reg': 0.0004239754565652459, 'random_strength': 5.466066104286921, 'bagging_temperature': 7.02428633344496, 'grow_policy': 'Lossguide'}. Best is trial 97 with value: 1017.3596815346203.


Best RMSE: 1017.3597
Best hyperparameters: {'learning_rate': 0.00904177511256763, 'depth': 6, 'l2_leaf_reg': 0.00021195181893025487, 'random_strength': 6.089756736211838, 'bagging_temperature': 7.330972112169229, 'grow_policy': 'Lossguide'}


# Using the best hyperparameters obtained from hyperparameter tuning by optuna to train the model

In [75]:
cbr_best = CatBoostRegressor(learning_rate=0.00561580529689733, 
                             depth=8, 
                             l2_leaf_reg=5.4291905312267685, 
                             random_strength= 3.7623938180796745, 
                             bagging_temperature=1.6862091427774766, 
                             grow_policy='Lossguide',
                             verbose=False)
cbr_best.fit(X_train, y_train)

y_pred = cbr_best.predict(X_val)

mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print(f'Root mean squared error: {rmse:.2f}')

Root mean squared error: 1018.90
